# Interactive Drone Control Parameter Tuning

Ce notebook permet de régler interactivement les paramètres du contrôle MPC du drone avec charge suspendue.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import expm
import cvxpy as cp
import ipywidgets as widgets
from IPython.display import display
import time

# Import classes from drone_control_1d module
from drone_control_1d import DroneSystem, MPCController, simulate_linear, simulate_nonlinear

%matplotlib inline

## Paramètres du système

Tous les paramètres sont regroupés dans cette section pour faciliter le réglage.

In [ ]:
# ============================================
# PARAMÈTRES - SECTION UNIQUE
# ============================================

# --- Paramètres de simulation ---
TS = 0.1  # Pas de temps (s)
HORIZON = 50  # Horizon de prédiction MPC (étapes)

# --- Conditions initiales ---
X_DRONE_INIT = 0.0  # Position initiale du drone (m)
V_DRONE_INIT = 10.0  # Vitesse initiale du drone (m/s)
THETA_INIT = -30.0  # Angle initial de la charge (degrés, négatif = derrière le drone)
OMEGA_INIT = 0.0  # Vitesse angulaire initiale (rad/s)

# --- Consigne ---
X_LOAD_TARGET = 10.0  # Position cible de la charge (m)

# --- Poids des états (fonction de coût) ---
Q_XD = 10.0  # Poids sur la position du drone
Q_VD = 20.0  # Poids sur la vitesse du drone
Q_THETA = 500.0  # Poids sur l'angle de la charge
Q_OMEGA = 100.0  # Poids sur la vitesse angulaire
Q_LOAD = 1000.0  # Poids sur la position de la charge

# --- Poids terminaux (fin de l'horizon) ---
QF_XD_MULT = 50.0  # Multiplicateur pour le poids terminal position drone
QF_VD_MULT = 50.0  # Multiplicateur pour le poids terminal vitesse drone
QF_THETA_MULT = 50.0  # Multiplicateur pour le poids terminal angle
QF_OMEGA_MULT = 50.0  # Multiplicateur pour le poids terminal vitesse angulaire
QF_LOAD_MULT = 100.0  # Multiplicateur pour le poids terminal position charge

# --- Autres paramètres ---
R_CONTROL = 2.0  # Poids sur l'effort de contrôle (pénalité accélération)

## Initialisation du système

In [ ]:
# Créer le système
system = DroneSystem()

print("Paramètres du système:")
print(f"  Masse du drone: {system.m_d} kg")
print(f"  Masse de la charge: {system.m_l} kg")
print(f"  Longueur du câble: {system.L} m")
print(f"  Surface de la charge: {system.S} m²")
print(f"  Coefficient de traînée: {system.Cd}")
print(f"  Pas de temps: {TS} s")

# Linéariser autour de l'équilibre
A, B = system.get_linearized_matrices()

# Discrétiser
Ad, Bd = system.discretize(A, B, TS)

print("\nSystème linéarisé et discrétisé avec succès")

## Fonction de simulation et visualisation

In [ ]:
def run_simulation_and_plot():
    """
    Exécute la simulation MPC en boucle ouverte et affiche les résultats
    pour les systèmes linéaire et non-linéaire.
    """
    # État initial (convertir l'angle en radians)
    theta_init_rad = np.deg2rad(THETA_INIT)
    x0 = np.array([X_DRONE_INIT, V_DRONE_INIT, theta_init_rad, OMEGA_INIT])
    
    # Position initiale de la charge
    x_load_init = x0[0] + system.L * np.sin(x0[2])
    
    print(f"État initial:")
    print(f"  Position drone: {x0[0]:.2f} m")
    print(f"  Vitesse drone: {x0[1]:.2f} m/s")
    print(f"  Angle charge: {np.rad2deg(x0[2]):.2f}°")
    print(f"  Vitesse angulaire: {x0[3]:.3f} rad/s")
    print(f"  Position charge: {x_load_init:.2f} m")
    print(f"\nCible: Position charge à {X_LOAD_TARGET:.2f} m")
    print(f"Horizon: {HORIZON} étapes ({HORIZON*TS:.1f} s)\n")
    
    # Poids de la fonction de coût
    Q_state = np.array([Q_XD, Q_VD, Q_THETA, Q_OMEGA])
    Qf_state = Q_state * np.array([QF_XD_MULT, QF_VD_MULT, QF_THETA_MULT, QF_OMEGA_MULT])
    Qf_load = Q_LOAD * QF_LOAD_MULT
    
    # Créer le contrôleur MPC
    controller = MPCController(Ad, Bd, system.L, horizon=HORIZON)
    
    # Résoudre le problème MPC (boucle ouverte)
    print("Résolution du problème MPC...")
    u_opt, x_opt, solve_time = controller.solve(
        x0, X_LOAD_TARGET, Q_state, Q_LOAD, R_CONTROL,
        Qf_state=Qf_state, Qf_load=Qf_load
    )
    
    if u_opt is None:
        print("ERREUR: L'optimisation a échoué!")
        return
    
    print(f"Optimisation résolue en {solve_time*1000:.2f} ms\n")
    
    # Simuler le système linéaire
    print("Simulation du système linéaire...")
    x_linear = simulate_linear(Ad, Bd, x0, u_opt)
    
    # Simuler le système non-linéaire
    print("Simulation du système non-linéaire...")
    t_nonlinear, x_nonlinear = simulate_nonlinear(system, x0, u_opt, TS)
    
    # Vecteur de temps pour les graphiques
    t = np.linspace(0, HORIZON * TS, HORIZON + 1)
    
    # Calculer la position de la charge
    x_load_linear = x_linear[0, :] + system.L * np.sin(x_linear[2, :])
    x_load_nonlinear = x_nonlinear[0, :] + system.L * np.sin(x_nonlinear[2, :])
    
    # Erreurs finales
    print("\nErreurs finales:")
    print(f"  Système linéaire:")
    print(f"    Position charge: {abs(x_load_linear[-1] - X_LOAD_TARGET):.4f} m")
    print(f"  Système non-linéaire:")
    print(f"    Position charge: {abs(x_load_nonlinear[-1] - X_LOAD_TARGET):.4f} m")
    
    # === GRAPHIQUES ===
    fig = plt.figure(figsize=(14, 10))
    
    # 1. Position de la charge (graphique principal)
    ax1 = plt.subplot(3, 2, 1)
    ax1.plot(t, x_load_linear, 'g--', label='Système linéaire', linewidth=2)
    ax1.plot(t, x_load_nonlinear, 'r-', label='Système non-linéaire', linewidth=2)
    ax1.axhline(X_LOAD_TARGET, color='k', linestyle=':', alpha=0.5, label='Cible')
    ax1.set_ylabel('Position charge (m)', fontsize=11)
    ax1.set_title('Position de la charge', fontsize=12, fontweight='bold')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # 2. Position du drone
    ax2 = plt.subplot(3, 2, 2)
    ax2.plot(t, x_linear[0, :], 'g--', label='Système linéaire', linewidth=2)
    ax2.plot(t, x_nonlinear[0, :], 'r-', label='Système non-linéaire', linewidth=2)
    ax2.set_ylabel('Position drone (m)', fontsize=11)
    ax2.set_title('Position du drone', fontsize=12, fontweight='bold')
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    # 3. Vitesse du drone
    ax3 = plt.subplot(3, 2, 3)
    ax3.plot(t, x_linear[1, :], 'g--', label='Système linéaire', linewidth=2)
    ax3.plot(t, x_nonlinear[1, :], 'r-', label='Système non-linéaire', linewidth=2)
    ax3.axhline(0, color='k', linestyle=':', alpha=0.5)
    ax3.set_ylabel('Vitesse drone (m/s)', fontsize=11)
    ax3.set_title('Vitesse du drone', fontsize=12, fontweight='bold')
    ax3.legend(loc='best')
    ax3.grid(True, alpha=0.3)
    
    # 4. Angle du câble
    ax4 = plt.subplot(3, 2, 4)
    ax4.plot(t, np.rad2deg(x_linear[2, :]), 'g--', label='Système linéaire', linewidth=2)
    ax4.plot(t, np.rad2deg(x_nonlinear[2, :]), 'r-', label='Système non-linéaire', linewidth=2)
    ax4.axhline(0, color='k', linestyle=':', alpha=0.5)
    ax4.set_ylabel('Angle câble (°)', fontsize=11)
    ax4.set_title('Angle du câble', fontsize=12, fontweight='bold')
    ax4.legend(loc='best')
    ax4.grid(True, alpha=0.3)
    
    # 5. Vitesse angulaire
    ax5 = plt.subplot(3, 2, 5)
    ax5.plot(t, x_linear[3, :], 'g--', label='Système linéaire', linewidth=2)
    ax5.plot(t, x_nonlinear[3, :], 'r-', label='Système non-linéaire', linewidth=2)
    ax5.axhline(0, color='k', linestyle=':', alpha=0.5)
    ax5.set_ylabel('Vitesse angulaire (rad/s)', fontsize=11)
    ax5.set_xlabel('Temps (s)', fontsize=11)
    ax5.set_title('Vitesse angulaire', fontsize=12, fontweight='bold')
    ax5.legend(loc='best')
    ax5.grid(True, alpha=0.3)
    
    # 6. Commande (accélération)
    ax6 = plt.subplot(3, 2, 6)
    t_u = t[:-1]
    ax6.step(t_u, u_opt[0, :], 'b-', where='post', linewidth=2, label='Commande MPC')
    ax6.axhline(0, color='k', linestyle=':', alpha=0.5)
    ax6.set_ylabel('Accélération drone (m/s²)', fontsize=11)
    ax6.set_xlabel('Temps (s)', fontsize=11)
    ax6.set_title('Commande de contrôle', fontsize=12, fontweight='bold')
    ax6.legend(loc='best')
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nSimulation terminée avec succès!")

## Exécution de la simulation

Exécutez la cellule suivante pour lancer la simulation avec les paramètres définis ci-dessus.

**Conditions initiales:**
- Position drone: 0 m
- Vitesse drone: 10 m/s
- Angle charge: 30° derrière le drone (angle négatif)
- Vitesse angulaire: 0 rad/s

**Consigne:** Position de la charge à 10 m

In [ ]:
# Lancer la simulation
run_simulation_and_plot()

## Interface interactive pour le réglage des paramètres

Utilisez les widgets ci-dessous pour ajuster les paramètres et observer l'effet sur le comportement du système.

In [ ]:
# Créer les widgets interactifs
style = {'description_width': '200px'}
layout = widgets.Layout(width='500px')

# Paramètres de simulation
w_ts = widgets.FloatSlider(
    value=TS, min=0.05, max=0.5, step=0.05,
    description='Pas de temps (s):',
    style=style, layout=layout
)

w_horizon = widgets.IntSlider(
    value=HORIZON, min=10, max=100, step=5,
    description='Horizon MPC (étapes):',
    style=style, layout=layout
)

# Conditions initiales
w_x_init = widgets.FloatSlider(
    value=X_DRONE_INIT, min=-10, max=10, step=0.5,
    description='Position drone initiale (m):',
    style=style, layout=layout
)

w_v_init = widgets.FloatSlider(
    value=V_DRONE_INIT, min=0, max=20, step=0.5,
    description='Vitesse drone initiale (m/s):',
    style=style, layout=layout
)

w_theta_init = widgets.FloatSlider(
    value=THETA_INIT, min=-45, max=45, step=1,
    description='Angle charge initial (°):',
    style=style, layout=layout
)

w_omega_init = widgets.FloatSlider(
    value=OMEGA_INIT, min=-1, max=1, step=0.05,
    description='Vitesse angulaire init. (rad/s):',
    style=style, layout=layout
)

# Consigne
w_target = widgets.FloatSlider(
    value=X_LOAD_TARGET, min=0, max=50, step=1,
    description='Position cible charge (m):',
    style=style, layout=layout
)

# Poids des états
w_q_xd = widgets.FloatLogSlider(
    value=Q_XD, base=10, min=-1, max=3, step=0.1,
    description='Poids Q_xd (position):',
    style=style, layout=layout
)

w_q_vd = widgets.FloatLogSlider(
    value=Q_VD, base=10, min=-1, max=3, step=0.1,
    description='Poids Q_vd (vitesse):',
    style=style, layout=layout
)

w_q_theta = widgets.FloatLogSlider(
    value=Q_THETA, base=10, min=0, max=4, step=0.1,
    description='Poids Q_theta (angle):',
    style=style, layout=layout
)

w_q_omega = widgets.FloatLogSlider(
    value=Q_OMEGA, base=10, min=0, max=3, step=0.1,
    description='Poids Q_omega (vit. ang.):',
    style=style, layout=layout
)

w_q_load = widgets.FloatLogSlider(
    value=Q_LOAD, base=10, min=0, max=4, step=0.1,
    description='Poids Q_load (pos. charge):',
    style=style, layout=layout
)

# Poids terminaux
w_qf_xd_mult = widgets.FloatSlider(
    value=QF_XD_MULT, min=1, max=200, step=5,
    description='Multiplicateur Qf_xd:',
    style=style, layout=layout
)

w_qf_vd_mult = widgets.FloatSlider(
    value=QF_VD_MULT, min=1, max=200, step=5,
    description='Multiplicateur Qf_vd:',
    style=style, layout=layout
)

w_qf_theta_mult = widgets.FloatSlider(
    value=QF_THETA_MULT, min=1, max=200, step=5,
    description='Multiplicateur Qf_theta:',
    style=style, layout=layout
)

w_qf_omega_mult = widgets.FloatSlider(
    value=QF_OMEGA_MULT, min=1, max=200, step=5,
    description='Multiplicateur Qf_omega:',
    style=style, layout=layout
)

w_qf_load_mult = widgets.FloatSlider(
    value=QF_LOAD_MULT, min=1, max=500, step=10,
    description='Multiplicateur Qf_load:',
    style=style, layout=layout
)

# Poids de contrôle
w_r_control = widgets.FloatLogSlider(
    value=R_CONTROL, base=10, min=-2, max=2, step=0.1,
    description='Poids R (contrôle):',
    style=style, layout=layout
)

# Bouton de mise à jour
button = widgets.Button(
    description='Mettre à jour et simuler',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px')
)

output = widgets.Output()

def on_button_click(b):
    """Callback pour le bouton de mise à jour"""
    global TS, HORIZON, X_DRONE_INIT, V_DRONE_INIT, THETA_INIT, OMEGA_INIT
    global X_LOAD_TARGET, Q_XD, Q_VD, Q_THETA, Q_OMEGA, Q_LOAD
    global QF_XD_MULT, QF_VD_MULT, QF_THETA_MULT, QF_OMEGA_MULT, QF_LOAD_MULT
    global R_CONTROL
    
    # Mettre à jour les paramètres globaux
    TS = w_ts.value
    HORIZON = w_horizon.value
    X_DRONE_INIT = w_x_init.value
    V_DRONE_INIT = w_v_init.value
    THETA_INIT = w_theta_init.value
    OMEGA_INIT = w_omega_init.value
    X_LOAD_TARGET = w_target.value
    Q_XD = w_q_xd.value
    Q_VD = w_q_vd.value
    Q_THETA = w_q_theta.value
    Q_OMEGA = w_q_omega.value
    Q_LOAD = w_q_load.value
    QF_XD_MULT = w_qf_xd_mult.value
    QF_VD_MULT = w_qf_vd_mult.value
    QF_THETA_MULT = w_qf_theta_mult.value
    QF_OMEGA_MULT = w_qf_omega_mult.value
    QF_LOAD_MULT = w_qf_load_mult.value
    R_CONTROL = w_r_control.value
    
    # Re-discrétiser si le pas de temps a changé
    global Ad, Bd
    Ad, Bd = system.discretize(A, B, TS)
    
    # Effacer l'output précédent et relancer la simulation
    with output:
        output.clear_output(wait=True)
        print("="*60)
        print("Mise à jour des paramètres et relance de la simulation...")
        print("="*60)
        run_simulation_and_plot()

button.on_click(on_button_click)

# Organiser les widgets
print("Interface interactive de réglage des paramètres")
print("="*60)

# Créer des sections
section_sim = widgets.VBox([
    widgets.HTML("<h3>Paramètres de simulation</h3>"),
    w_ts, w_horizon
])

section_init = widgets.VBox([
    widgets.HTML("<h3>Conditions initiales</h3>"),
    w_x_init, w_v_init, w_theta_init, w_omega_init
])

section_target = widgets.VBox([
    widgets.HTML("<h3>Consigne</h3>"),
    w_target
])

section_weights = widgets.VBox([
    widgets.HTML("<h3>Poids des états (fonction de coût)</h3>"),
    w_q_xd, w_q_vd, w_q_theta, w_q_omega, w_q_load
])

section_terminal = widgets.VBox([
    widgets.HTML("<h3>Poids terminaux (multiplicateurs)</h3>"),
    w_qf_xd_mult, w_qf_vd_mult, w_qf_theta_mult, w_qf_omega_mult, w_qf_load_mult
])

section_control = widgets.VBox([
    widgets.HTML("<h3>Autres paramètres</h3>"),
    w_r_control
])

# Afficher tout
display(section_sim)
display(section_init)
display(section_target)
display(section_weights)
display(section_terminal)
display(section_control)
display(button)
display(output)

## Notes d'utilisation

### Paramètres clés:

1. **Horizon MPC**: Plus l'horizon est long, meilleure est la prévision, mais le temps de calcul augmente.

2. **Poids Q_load**: Détermine l'importance de faire atteindre la position cible à la charge. Des valeurs élevées forcent un suivi précis.

3. **Poids Q_theta**: Contrôle l'importance de garder l'angle du câble proche de la verticale. Des valeurs élevées réduisent l'oscillation.

4. **Poids R**: Pénalise l'effort de contrôle. Des valeurs élevées donnent des commandes plus douces mais une convergence plus lente.

5. **Poids terminaux**: Multiplicateurs appliqués aux poids des états à la fin de l'horizon. Ils encouragent le système à atteindre l'état cible à la fin de l'horizon.

### Observations:

- Le **système linéaire** (ligne verte) utilise des approximations aux petits angles.
- Le **système non-linéaire** (ligne rouge) inclut la dynamique complète avec résistance de l'air.
- Les deux systèmes peuvent diverger pour de grandes perturbations initiales ou des angles importants.

### Conditions initiales du problème:

- Position drone: **0 m**
- Vitesse drone: **10 m/s**
- Angle de la charge: **-30°** (derrière le drone)
- Vitesse angulaire: **0 rad/s**
- **Cible**: Position de la charge à **10 m**